In [1]:
import sys
sys.path.append('../../')
from utils.packages import *
from utils.ml_fairness import *
from utils.standard_data import *
dir = 'res/titanic/'
Path(dir).mkdir(parents=True, exist_ok=True)

d_fields = ['Name', 'Stage', 'CVR', 'CVD', 'V_SPD', 'V_EOD', 'V_AOD', 'V_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
diff_file = dir + 'diff' + '.csv'
if(not os.path.isfile(diff_file)):
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(d_fields)
    
f_count = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name)) and not name.startswith('.')])
fields = ['Acc', 'F1', 'DI','SPD', 'EOD', 'AOD', 'ERD', 'CNT', 'TI']
filename = dir + str(f_count) + '.csv'
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)

In [2]:
# Load data
train = pd.read_csv('../../data/titanic/train.csv')
test = pd.read_csv('../../data/titanic/test.csv')
df = train

In [3]:
## BASIC PREP
df['Sex'] = df['Sex'].replace({'female': 0.0, 'male': 1.0})

### Missing value
df['Age'].fillna(df['Age'].median(), inplace = True)
#complete embarked with mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
#complete missing fare with median
df['Fare'].fillna(df['Fare'].median(), inplace = True)

### Feature engineering

## Drop feature
drop_column = ['PassengerId','Cabin', 'Ticket', 'Name', 'Parch']

## generation
df['FamilySize'] = df ['SibSp'] + df['Parch'] + 1
df['IsAlone'] = 1 #initialize to yes/1 is alone
df['IsAlone'].loc[df['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1
df['Title'] = df['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (df['Title'].value_counts() < stat_min) #this will create a true false series with title name as index
df['Title'] = df['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)

## Binning
df['Fare'] = pd.qcut(df['Fare'], 4)
df['Age'] = pd.cut(df['Age'].astype(int), 5)

### Encoding
## LabelEncoder
label = LabelEncoder()
df['Embarked'] = label.fit_transform(df['Embarked'])
df['Title'] = label.fit_transform(df['Title'])
df['Age'] = label.fit_transform(df['Age'])
df['Fare'] = label.fit_transform(df['Fare'])

## One-hot encoder
# cat_feat = ['Cabin', 'Ticket', 'Embarked']
# df = pd.get_dummies(df, columns=cat_feat, prefix_sep='=')

df.drop(drop_column, axis=1, inplace = True)

In [4]:
seed = 42 # randrange(100)
y2_train, y2_test = train_test_split(df, test_size = 0.3, random_state = seed) # stratify=df['loan']
y1_train, y1_test = train_test_split(df, test_size = 0.3, random_state = seed) # 

In [5]:
pro_att_name = ['Sex']
priv_class = [1]
reamining_cat_feat = []

y2_data_orig_train, y2_X_train, y2_y_train = load_titanic_data(df, pro_att_name, priv_class, reamining_cat_feat)
y2_data_orig_test, y2_X_test, y2_y_test = load_titanic_data(y2_test, pro_att_name, priv_class, reamining_cat_feat)

y1_data_orig_train, y1_X_train, y1_y_train = load_titanic_data(df, pro_att_name, priv_class, reamining_cat_feat)
y1_data_orig_test, y1_X_test, y1_y_test = load_titanic_data(y1_test, pro_att_name, priv_class, reamining_cat_feat)


In [6]:
from xgboost import XGBClassifier

y2_xgb = XGBClassifier(learning_rate= 0.01, max_depth= 4, n_estimators= 300, seed= 0)
y2_mdl = y2_xgb.fit(y2_X_train, y2_y_train)

y1_xgb = XGBClassifier(learning_rate= 0.01, max_depth= 4, n_estimators= 300, seed= 0)
y1_mdl = y1_xgb.fit(y1_X_train, y1_y_train)

In [7]:
# plot_model_performance(y2_mdl, y2_X_test, y2_y_test)
y1_pred, y1_fair = get_fair_metrics_and_plot(filename, y1_data_orig_test, y1_mdl)
y2_pred, y2_fair = get_fair_metrics_and_plot(filename, y2_data_orig_test, y2_mdl)

In [8]:
y1_fair = y1_fair.drop(['DI', 'CNT', 'TI'], axis=1)
y2_fair = y2_fair.drop(['DI', 'CNT', 'TI'], axis=1)
CVR, CVD, AVR_EOD, AVD_EOD, AVR_SPD, AVD_SPD, AVD_AOD, AV_ERD = compute_new_metrics(y2_data_orig_test, y1_pred, y2_pred)
row_y1 = y1_fair.iloc[[0]].values[0].tolist()
row_y2 = y2_fair.iloc[[0]].values[0].tolist()
diff = []

diff.append(CVR)
diff.append(CVD)
diff.append(AVD_SPD)
diff.append(AVD_EOD)
diff.append(AVD_AOD)
diff.append(AV_ERD)

for i in range(len(row_y2)):
    if(i < 2):
        change = row_y2[i] - row_y1[i]
    else:
        sign = ''
        if(row_y2[i] >= 0 and row_y1[i] >= 0):
            sign = '(+)'
            d = abs(row_y2[i]) - abs(row_y1[i])
        if(row_y2[i] < 0 and row_y1[i] < 0):
            sign = '(-)'
            d = abs(row_y2[i]) - abs(row_y1[i])
        if(row_y2[i] < 0 and row_y1[i] >= 0):
            sign = '(+-)'
            d = row_y2[i] - row_y1[i]
        if(row_y2[i] >=0 and row_y1[i] < 0):
            sign = '(-+)'
            d = row_y2[i] - row_y1[i]
        d = round(d, 3)
        change = sign + ' ' + str(d)

    diff.append(change)

cols = ['CVR', 'CVD ', 'AV_SPD', 'AV_EOD', 'AV_AOD', 'AV_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
# metrics = pd.DataFrame(data=obj_fairness, index=['y1'], columns=cols)
diff_df = pd.DataFrame(data=[diff], columns  = cols, index = ['Diff']).round(3)
stage = 'NOthing'
model_name = 'titanic-baseline'
diff = diff_df.iloc[0].values.tolist()
diff.insert(0, stage)
diff.insert(0, model_name)
with open(diff_file, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(diff)    
    diff_df

Unprinv: Sex 0.0
